<a href="https://colab.research.google.com/github/Eshita781996/SCD/blob/main/SCD4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip -q install findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('delta_session').getOrCreate()

In [ ]:
# employees_df = spark.createDataFrame([(1, 'John', 'Sydney', 35000.00),(2, 'Peter', 'Melbourne', 45000.00),(3, 'Sam', 'Sydney',55000.00)],
#                                      schema='emp_id int,emp_name string,emp_city string,emp_salary float')

employees_delta_df = spark.createDataFrame([(2, 'Peter', 'Melbourne', 25005.00),(5, 'Jessie', 'Brisbane', 23004.0)],
                                           schema='emp_id int,emp_name string,emp_city string,emp_salary float')

In [ ]:
from delta.tables import *
if not DeltaTable.isDeltaTable(spark,'employees'):
  employees_delta_df.write.format("delta").save("employees")
  if not DeltaTable.isDeltaTable(spark,'employees_history'):
    employees_delta_df = employees_delta_df.withColumn('create_Date',current_timestamp())
    employees_delta_df.write.format("delta").save("employees_history")
else:
  deltaTable = DeltaTable.forPath(spark, "employees")
  deltaTable.alias("history").merge(
  employees_delta_df.alias("updates"),
  "history.emp_id = updates.emp_id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

  if not DeltaTable.isDeltaTable(spark,'employees_history'):
    employees_delta_df = employees_delta_df.withColumn('create_Date',from_utc_timestamp(current_timestamp(),'IST'))
    employees_delta_df.write.format("delta").save("employees_history")
  else:
    df= spark.sql("select INPUT_VIEW.*,from_utc_timestamp(current_timestamp(),'IST') as create_Date \
      from INPUT_VIEW join employees_history on employees_history.emp_id=INPUT_VIEW.emp_id \
      where  employees_history.emp_salary<>INPUT_VIEW.emp_salary ")
    df.write.format("delta").mode('append').option("mergeSchema", "true").save("employees_history")

In [ ]:
spark.read.format("delta").load("employees").show()
spark.read.format("delta").load("employees_history").show()

+------+--------+---------+----------+
|emp_id|emp_name| emp_city|emp_salary|
+------+--------+---------+----------+
|     2|   Peter|Melbourne|   25005.0|
|     5|  Jessie| Brisbane|   23004.0|
+------+--------+---------+----------+

+------+--------+---------+----------+--------------------+
|emp_id|emp_name| emp_city|emp_salary|         create_Date|
+------+--------+---------+----------+--------------------+
|     2|   Peter|Melbourne|   25005.0|2024-01-05 00:26:...|
|     2|   Peter|Melbourne|   25005.0|2024-01-05 00:26:...|
|     5|  Jessie| Brisbane|   23004.0|2024-01-05 00:26:...|
|     2|   Peter|Melbourne|   35005.0|2024-01-05 00:25:...|
|     5|  Jessie| Brisbane|   43004.0|2024-01-04 22:58:...|
|     2|   Peter|Melbourne|   35002.0|2024-01-04 22:58:...|
+------+--------+---------+----------+--------------------+

